# Chemical Reaction Prediction with Transformers (2025 Edition)

![Molecular Transformer Animation](https://pubs.acs.org/cms/10.1021/acscentsci.9b00576/asset/images/medium/oc9b00576_0009.gif)
<center><b>Figure 1:</b> SMILES-to-SMILES translation with the Molecular Transformer</center>

---

## Course: 06-731 Molecular Machine Learning
**Original lecture by:** Philippe Schwaller (IBM Research / EPFL)  
**Updated for 2025 by:** Gomes Group, Carnegie Mellon University

---

## What's New in This Version?

This notebook is an updated version of the 2022 "Digital Molecular Design Studio" notebook on Chemical Language Models. Key changes include:

| Component | 2022 Version | 2025 Version |
|-----------|--------------|---------------|
| **Framework** | OpenNMT-py 2.2.0 | HuggingFace Transformers (ReactionT5v2) |
| **Model** | Custom Molecular Transformer | Pre-trained ReactionT5v2 |
| **Python** | 3.6-3.8 | 3.10-3.12 |
| **Training** | From scratch (24+ hours) | Fine-tuning (minutes) or inference-only |
| **Atom Mapping** | Not included | RXNMapper integration |
| **API Access** | Not available | rxn4chemistry (IBM RXN) |

### Why the Change?

1. **OpenNMT-py is deprecated** - The project announced it's "no longer actively supported" as of July 2024, with [Eole](https://github.com/eole-nlp/eole) as the successor.
2. **pyonmttok dependency issues** - The tokenizer package fails on Python 3.12 (Google Colab's current version).
3. **Better alternatives exist** - ReactionT5v2 offers state-of-the-art performance with easier setup.

---

## Table of Contents

### Part 1: Setup and Foundations
- [1.1 Environment Setup](#1.1)
- [1.2 SMILES Representation Review](#1.2)
- [1.3 Tokenization for Chemical Language Models](#1.3)

### Part 2: Reaction Prediction with ReactionT5v2
- [2.1 Loading Pre-trained Models](#2.1)
- [2.2 Forward Reaction Prediction](#2.2)
- [2.3 Batch Predictions and Evaluation](#2.3)
- [2.4 Retrosynthesis Prediction](#2.4)

### Part 3: Advanced Topics
- [3.1 Atom Mapping with RXNMapper](#3.1)
- [3.2 IBM RXN for Chemistry API](#3.2)
- [3.3 Data Augmentation for SMILES](#3.3)
- [3.4 Fine-tuning on Custom Datasets](#3.4)

### Part 4: Visualization and Analysis
- [4.1 Drawing Chemical Reactions](#4.1)
- [4.2 Error Analysis and Debugging](#4.2)

### Appendix
- [A. Legacy OpenNMT-py Approach (Reference Only)](#appendix-a)
- [B. Publications and Resources](#appendix-b)

---

## Part 1: Setup and Foundations

<a id='1.1'></a>
### 1.1 Environment Setup

We'll install the necessary packages for this notebook. The installation is designed to work on Google Colab (Python 3.12) and local conda environments.

In [ ]:
# Cell 1: Environment Detection and Package Installation
import sys
import subprocess

IN_COLAB = 'google.colab' in sys.modules
print(f"Running in Google Colab: {IN_COLAB}")
print(f"Python version: {sys.version}")

# Core packages for chemical reaction prediction
PACKAGES = [
    "rdkit",                    # Cheminformatics toolkit
    "torch",                    # PyTorch (usually pre-installed in Colab)
    "transformers>=4.40.0",     # HuggingFace Transformers
    "tokenizers>=0.19.1",       # Fast tokenizers
    "sentencepiece",            # Tokenizer backend
    "accelerate",               # Training acceleration
    "datasets",                 # HuggingFace datasets
    "pandas",                   # Data manipulation
    "gdown",                    # Google Drive downloads
    "tqdm",                     # Progress bars
    "plotly",                   # Interactive visualization
]

def install_packages(packages):
    """Install packages with progress feedback."""
    for pkg in packages:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
    print("\n[SUCCESS] All packages installed!")

if IN_COLAB:
    install_packages(PACKAGES)

In [ ]:
# Cell 2: Import Libraries and Configure Environment
import os
import re
import random
import warnings
from typing import List, Dict, Optional, Tuple

import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm

# RDKit imports
from rdkit import Chem
from rdkit.Chem import Draw, AllChem, rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D, IPythonConsole
from rdkit import RDLogger

# HuggingFace imports
from transformers import AutoTokenizer, T5ForConditionalGeneration

# Display settings
IPythonConsole.ipython_useSVG = True
RDLogger.DisableLog('rdApp.*')  # Suppress RDKit warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Check GPU availability
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\nUsing device: {DEVICE}")
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print(f"\nRDKit version: {Chem.rdBase.rdkitVersion}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Cell 3: Gomes Group Color Palette for Visualizations
# Professional styling consistent with Gomes Group publications

GOMES_COLORS = {
    'teal': '#00D9FF',      # Primary accent
    'coral': '#FF6B6B',     # Secondary accent / errors
    'navy': '#0A1628',      # Dark background
    'slate': '#2D3748',     # Text
    'success': '#48BB78',   # Correct predictions
    'warning': '#ECC94B',   # Warnings
    'light_bg': '#F7FAFC',  # Light background
}

# Plotly template for consistent styling
import plotly.graph_objects as go
import plotly.express as px

GOMES_TEMPLATE = go.layout.Template(
    layout=go.Layout(
        font=dict(family="Helvetica Neue, Arial, sans-serif", size=14, color=GOMES_COLORS['slate']),
        plot_bgcolor=GOMES_COLORS['light_bg'],
        paper_bgcolor='white',
        colorway=[GOMES_COLORS['teal'], GOMES_COLORS['coral'], GOMES_COLORS['success'], GOMES_COLORS['warning']],
        title=dict(font=dict(size=20, color=GOMES_COLORS['navy'])),
    )
)

print("Gomes Group styling configured.")

<a id='1.2'></a>
### 1.2 SMILES Representation Review

**SMILES (Simplified Molecular Input Line Entry System)** is a line notation for representing molecules as text strings. For reaction prediction, we treat chemical reactions as a translation problem:

```
Reactants.Reagents >> Products
```

Key concepts:
- **Atoms**: C, N, O, S, etc. (uppercase = aromatic, lowercase = aromatic ring member)
- **Bonds**: Single (implicit or `-`), double (`=`), triple (`#`)
- **Rings**: Numbers indicate ring closures (e.g., `C1CCCCC1` = cyclohexane)
- **Branches**: Parentheses for branching (e.g., `CC(C)C` = isobutane)
- **Stereochemistry**: `@`, `@@` for chirality; `/`, `\` for cis/trans

In [ ]:
# Cell 4: SMILES Examples and Canonicalization

def canonicalize_smiles(smiles: str, verbose: bool = False) -> str:
    """
    Convert a SMILES string to its canonical form.
    
    Canonicalization ensures that the same molecule always has the same SMILES,
    regardless of how it was originally written. This is crucial for:
    1. Comparing predictions to ground truth
    2. Removing duplicates from datasets
    3. Ensuring reproducibility
    
    Args:
        smiles: Input SMILES string
        verbose: Print warning for invalid SMILES
    
    Returns:
        Canonical SMILES or empty string if invalid
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return Chem.MolToSmiles(mol)
        else:
            if verbose:
                print(f"Warning: '{smiles}' could not be parsed as a valid molecule.")
            return ''
    except Exception as e:
        if verbose:
            print(f"Error processing '{smiles}': {e}")
        return ''

# Demonstrate canonicalization
examples = [
    ("C(C)O", "Ethanol - different atom ordering"),
    ("OCC", "Ethanol - starting from oxygen"),
    ("CCO", "Ethanol - canonical form"),
    ("c1ccccc1", "Benzene - aromatic notation"),
    ("C1=CC=CC=C1", "Benzene - Kekule notation"),
]

print("SMILES Canonicalization Examples:")
print("-" * 60)
for smiles, description in examples:
    canonical = canonicalize_smiles(smiles)
    print(f"{smiles:20} -> {canonical:20} ({description})")

In [ ]:
# Cell 5: Visualize molecules from SMILES

def draw_molecules_grid(smiles_list: List[str], labels: Optional[List[str]] = None, 
                        mols_per_row: int = 4, img_size: Tuple[int, int] = (250, 200)):
    """
    Draw a grid of molecules from SMILES strings.
    
    Args:
        smiles_list: List of SMILES strings
        labels: Optional labels for each molecule
        mols_per_row: Number of molecules per row
        img_size: Size of each molecule image
    
    Returns:
        RDKit grid image
    """
    mols = [Chem.MolFromSmiles(smi) for smi in smiles_list]
    if labels is None:
        labels = smiles_list
    
    return Draw.MolsToGridImage(
        mols, 
        molsPerRow=mols_per_row, 
        subImgSize=img_size,
        legends=labels
    )

# Example: Common functional groups in drug molecules
drug_motifs = [
    ("c1ccccc1", "Benzene"),
    ("c1ccncc1", "Pyridine"),
    ("c1ccc2[nH]ccc2c1", "Indole"),
    ("CC(=O)O", "Acetic acid"),
    ("CC(=O)N", "Acetamide"),
    ("c1ccc(O)cc1", "Phenol"),
    ("c1ccc(N)cc1", "Aniline"),
    ("CC(C)C", "Isobutane"),
]

smiles_list, labels = zip(*drug_motifs)
draw_molecules_grid(list(smiles_list), list(labels))

<a id='1.3'></a>
### 1.3 Tokenization for Chemical Language Models

Unlike natural language, SMILES requires **atom-wise tokenization** to preserve chemical meaning. We use a regex pattern that recognizes:

- Multi-character elements (Br, Cl)
- Bracketed atoms ([NH2], [Fe+2])
- Ring numbers (including extended %XX notation)
- Stereochemistry markers
- Special symbols (reaction arrow `>>`, etc.)

In [ ]:
# Cell 6: SMILES Tokenization

# Regex pattern from the original Molecular Transformer paper
SMI_REGEX_PATTERN = r"(\%\([0-9]{3}\)|\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\||\(|\)|\.|=|#|-|\+|\\|\/|:|~|@|\?|>>?|\*|\$|\%[0-9]{2}|[0-9])"

def smiles_tokenizer(smiles: str) -> str:
    """
    Tokenize a SMILES string into space-separated tokens.
    
    This tokenization preserves chemical meaning by keeping:
    - Multi-character elements together (Br, Cl)
    - Bracketed atoms as single tokens ([NH2])
    - Ring numbers as single tokens
    
    Args:
        smiles: Input SMILES string
    
    Returns:
        Space-separated tokenized SMILES
    """
    regex = re.compile(SMI_REGEX_PATTERN)
    tokens = regex.findall(smiles)
    return ' '.join(tokens)

def detokenize_smiles(tokenized: str) -> str:
    """Convert tokenized SMILES back to standard SMILES."""
    return tokenized.replace(' ', '')

# Demonstration
test_smiles = [
    "CCO",                                    # Simple: ethanol
    "CC(=O)Oc1ccccc1C(=O)O",                  # Aspirin
    "[NH2]c1ccccc1Br",                        # Bracketed atoms
    "CC.CC>>CCCC",                            # Reaction SMILES
    "C[C@@H](O)CC",                           # Stereochemistry
]

print("SMILES Tokenization Examples:")
print("=" * 70)
for smi in test_smiles:
    tokenized = smiles_tokenizer(smi)
    n_tokens = len(tokenized.split())
    print(f"\nOriginal ({len(smi)} chars):")
    print(f"  {smi}")
    print(f"Tokenized ({n_tokens} tokens):")
    print(f"  {tokenized}")

---

## Part 2: Reaction Prediction with ReactionT5v2

<a id='2.1'></a>
### 2.1 Loading Pre-trained Models

**ReactionT5v2** is a T5 model pre-trained on the [Open Reaction Database (ORD)](https://github.com/open-reaction-database/ord-data), which contains millions of diverse chemical reactions. Unlike models trained on narrow patent datasets, ReactionT5 offers:

- **Greater generalizability** across reaction types
- **State-of-the-art performance** on standard benchmarks
- **Easy fine-tuning** for specialized applications

Available models on [HuggingFace](https://huggingface.co/collections/sagawa/reactiont5-67dbe0550cbb6886a85e348b):
- `sagawa/ReactionT5v2-forward` - Forward reaction prediction (reactants -> products)
- `sagawa/ReactionT5v2-retrosynthesis` - Retrosynthesis (products -> reactants)
- `sagawa/ReactionT5v2-yield` - Yield prediction

In [ ]:
# Cell 7: Load ReactionT5v2 Forward Model

class ReactionT5Predictor:
    """
    Wrapper class for ReactionT5v2 models providing easy-to-use prediction methods.
    
    This class handles:
    - Model and tokenizer loading
    - Input formatting (REACTANT:...REAGENT:... format)
    - Batch prediction with beam search
    - Output post-processing and validation
    """
    
    def __init__(self, model_name: str = "sagawa/ReactionT5v2-forward", device: str = None):
        """
        Initialize the predictor with a pre-trained model.
        
        Args:
            model_name: HuggingFace model identifier
            device: 'cuda' or 'cpu' (auto-detected if None)
        """
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model_name = model_name
        
        print(f"Loading model: {model_name}")
        print(f"Device: {self.device}")
        
        # Load tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.model.to(self.device)
        self.model.eval()
        
        print(f"Model loaded successfully!")
        print(f"Parameters: {sum(p.numel() for p in self.model.parameters()):,}")
    
    def format_input(self, reactants: str, reagents: str = " ") -> str:
        """
        Format input for ReactionT5 models.
        
        Expected format: "REACTANT:{smiles}REAGENT:{smiles}"
        If no reagents, use a space as placeholder.
        
        Args:
            reactants: SMILES of reactants (concatenated with '.')
            reagents: SMILES of reagents/catalysts/solvents (or ' ')
        
        Returns:
            Formatted input string
        """
        return f"REACTANT:{reactants}REAGENT:{reagents}"
    
    def predict(self, reactants: str, reagents: str = " ", 
                num_beams: int = 5, num_return_sequences: int = 5,
                max_length: int = 200) -> List[Dict]:
        """
        Predict products for a single reaction.
        
        Args:
            reactants: SMILES of reactants
            reagents: SMILES of reagents (default: ' ' for none)
            num_beams: Beam search width
            num_return_sequences: Number of predictions to return
            max_length: Maximum output sequence length
        
        Returns:
            List of dicts with 'smiles', 'canonical', 'valid', 'score' keys
        """
        # Format and tokenize input
        input_text = self.format_input(reactants, reagents)
        inputs = self.tokenizer(input_text, return_tensors="pt", padding=True)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # Generate predictions
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                num_beams=num_beams,
                num_return_sequences=num_return_sequences,
                max_length=max_length,
                return_dict_in_generate=True,
                output_scores=True,
            )
        
        # Decode and validate predictions
        results = []
        for seq in outputs.sequences:
            pred_smiles = self.tokenizer.decode(seq, skip_special_tokens=True)
            pred_smiles = pred_smiles.replace(' ', '').rstrip('.')
            canonical = canonicalize_smiles(pred_smiles)
            
            results.append({
                'smiles': pred_smiles,
                'canonical': canonical,
                'valid': canonical != '',
            })
        
        return results
    
    def predict_batch(self, reactions: List[Dict], batch_size: int = 16,
                      num_beams: int = 5, num_return_sequences: int = 5) -> List[List[Dict]]:
        """
        Batch prediction for multiple reactions.
        
        Args:
            reactions: List of dicts with 'reactants' and optionally 'reagents' keys
            batch_size: Batch size for inference
            num_beams: Beam search width
            num_return_sequences: Predictions per reaction
        
        Returns:
            List of prediction lists (one per input reaction)
        """
        all_results = []
        
        for i in tqdm(range(0, len(reactions), batch_size), desc="Predicting"):
            batch = reactions[i:i+batch_size]
            
            # Format inputs
            input_texts = [
                self.format_input(r['reactants'], r.get('reagents', ' '))
                for r in batch
            ]
            
            inputs = self.tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    num_beams=num_beams,
                    num_return_sequences=num_return_sequences,
                    max_length=200,
                )
            
            # Decode outputs
            decoded = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
            
            # Group by input reaction
            for j in range(len(batch)):
                start_idx = j * num_return_sequences
                end_idx = start_idx + num_return_sequences
                
                preds = []
                for pred in decoded[start_idx:end_idx]:
                    pred = pred.replace(' ', '').rstrip('.')
                    canonical = canonicalize_smiles(pred)
                    preds.append({
                        'smiles': pred,
                        'canonical': canonical,
                        'valid': canonical != '',
                    })
                all_results.append(preds)
        
        return all_results

# Initialize the predictor
print("Initializing ReactionT5v2 Forward Predictor...")
print("(This may take a moment to download the model)")
print()

forward_predictor = ReactionT5Predictor("sagawa/ReactionT5v2-forward", device=DEVICE)

<a id='2.2'></a>
### 2.2 Forward Reaction Prediction

Let's test the model on some example reactions. We'll predict products from reactants and reagents.

In [ ]:
# Cell 8: Single Reaction Prediction Examples

# Example reactions with known products
test_reactions = [
    {
        "name": "Esterification (Fischer)",
        "reactants": "CC(=O)O.CCO",  # Acetic acid + Ethanol
        "reagents": " ",  # Acid catalyst (implicit)
        "expected": "CCOC(C)=O",  # Ethyl acetate
    },
    {
        "name": "Suzuki Coupling",
        "reactants": "Brc1ccccc1.OB(O)c1ccccc1",  # Bromobenzene + Phenylboronic acid
        "reagents": "[Pd]",  # Palladium catalyst
        "expected": "c1ccc(-c2ccccc2)cc1",  # Biphenyl
    },
    {
        "name": "Amide Formation",
        "reactants": "CC(=O)O.NCC",  # Acetic acid + Ethylamine
        "reagents": " ",
        "expected": "CCNC(C)=O",  # N-ethylacetamide
    },
    {
        "name": "Grignard Addition",
        "reactants": "CC=O.[Mg]Br",  # Acetaldehyde + MgBr
        "reagents": "CCCC",  # Butyl (as Grignard)
        "expected": "CCCCC(C)O",  # 2-hexanol
    },
]

print("Forward Reaction Prediction Examples")
print("=" * 70)

for rxn in test_reactions:
    print(f"\n{rxn['name']}")
    print("-" * 50)
    print(f"Reactants: {rxn['reactants']}")
    print(f"Reagents:  {rxn['reagents']}")
    print(f"Expected:  {rxn['expected']}")
    
    # Get predictions
    predictions = forward_predictor.predict(
        rxn['reactants'], 
        rxn['reagents'],
        num_beams=5,
        num_return_sequences=3
    )
    
    print("\nPredictions:")
    expected_canonical = canonicalize_smiles(rxn['expected'])
    
    for i, pred in enumerate(predictions, 1):
        status = "[CORRECT]" if pred['canonical'] == expected_canonical else ""
        valid = "valid" if pred['valid'] else "INVALID"
        print(f"  {i}. {pred['canonical']:40} ({valid}) {status}")

In [ ]:
# Cell 9: Visualize a Predicted Reaction
from IPython.display import SVG, display, HTML

def draw_chemical_reaction(rxn_smiles: str, highlight_by_reactant: bool = True,
                           use_smiles: bool = True, width: int = 800, height: int = 250) -> str:
    """
    Draw a chemical reaction from SMILES.
    
    Args:
        rxn_smiles: Reaction SMILES (format: reactants>>products)
        highlight_by_reactant: Color atoms by which reactant they came from
        use_smiles: Parse as SMILES (True) or SMARTS (False)
        width: Image width
        height: Image height
    
    Returns:
        SVG string of the reaction drawing
    """
    try:
        rxn = rdChemReactions.ReactionFromSmarts(rxn_smiles, useSmiles=use_smiles)
        d2d = rdMolDraw2D.MolDraw2DSVG(width, height)
        d2d.drawOptions().annotationFontScale = 1.2
        d2d.DrawReaction(rxn, highlightByReactant=highlight_by_reactant)
        d2d.FinishDrawing()
        return d2d.GetDrawingText()
    except Exception as e:
        return f"<p>Error drawing reaction: {e}</p>"

# Draw the first test reaction with its prediction
rxn = test_reactions[0]
predictions = forward_predictor.predict(rxn['reactants'], rxn['reagents'], num_return_sequences=1)
pred_smiles = predictions[0]['canonical']

print(f"Reaction: {rxn['name']}")
print(f"Input: {rxn['reactants']}")
print(f"Predicted Product: {pred_smiles}")
print(f"Expected Product: {rxn['expected']}")

rxn_smiles = f"{rxn['reactants']}>>{pred_smiles}"
display(SVG(draw_chemical_reaction(rxn_smiles)))

<a id='2.3'></a>
### 2.3 Batch Predictions and Evaluation

Now let's evaluate the model on a larger dataset. We'll download a subset of the USPTO dataset and compute standard metrics (Top-1, Top-3, Top-5 accuracy).

In [ ]:
# Cell 10: Download USPTO Dataset Subset
import gdown

def download_uspto_data(data_dir: str = "USPTO_480k") -> None:
    """
    Download USPTO 480k reaction dataset.
    
    This dataset contains ~480,000 reactions from US patents,
    split into train/val/test sets.
    """
    os.makedirs(data_dir, exist_ok=True)
    
    # Google Drive links for USPTO_480k
    files = [
        ("https://drive.google.com/uc?id=1RysNBvB2rsMP0Ap9XXi02XiiZkEXCrA8", "src-train.txt"),
        ("https://drive.google.com/uc?id=1CxxcVqtmOmHE2nhmqPFA6bilavzpcIlb", "tgt-train.txt"),
        ("https://drive.google.com/uc?id=1FFN1nz2yB4VwrpWaBuiBDzFzdX3ONBsy", "src-val.txt"),
        ("https://drive.google.com/uc?id=1pYCjWkYvgp1ZQ78EKQBArOvt_2P1KnmI", "tgt-val.txt"),
        ("https://drive.google.com/uc?id=10t6pHj9yR8Tp3kDvG0KMHl7Bt_TUbQ8W", "src-test.txt"),
        ("https://drive.google.com/uc?id=1FeGuiGuz0chVBRgePMu0pGJA4FVReA-b", "tgt-test.txt"),
    ]
    
    for url, filename in files:
        target = os.path.join(data_dir, filename)
        if not os.path.exists(target):
            print(f"Downloading {filename}...")
            gdown.download(url, target, quiet=False)
        else:
            print(f"{filename} already exists.")

def load_uspto_data(data_dir: str = "USPTO_480k", split: str = "val", 
                    max_samples: int = None) -> pd.DataFrame:
    """
    Load USPTO dataset into a DataFrame.
    
    Args:
        data_dir: Directory containing the data files
        split: 'train', 'val', or 'test'
        max_samples: Maximum number of samples to load (None for all)
    
    Returns:
        DataFrame with 'precursors' and 'products' columns
    """
    src_file = os.path.join(data_dir, f"src-{split}.txt")
    tgt_file = os.path.join(data_dir, f"tgt-{split}.txt")
    
    with open(src_file, 'r') as f:
        precursors = [line.strip().replace(' ', '') for line in f]
    with open(tgt_file, 'r') as f:
        products = [line.strip().replace(' ', '') for line in f]
    
    df = pd.DataFrame({'precursors': precursors, 'products': products})
    
    if max_samples:
        df = df.head(max_samples)
    
    return df

# Download and load data
print("Downloading USPTO 480k dataset...")
download_uspto_data()

# For demonstration, we'll use a smaller subset
N_EVAL_SAMPLES = 100  # Use 100 samples for quick evaluation
val_df = load_uspto_data(split="val", max_samples=N_EVAL_SAMPLES)

print(f"\nLoaded {len(val_df)} validation reactions")
print("\nSample reactions:")
val_df.head()

In [ ]:
# Cell 11: Evaluate Model on USPTO Validation Set

def evaluate_predictions(df: pd.DataFrame, predictor: ReactionT5Predictor,
                        n_best: int = 5, batch_size: int = 16) -> pd.DataFrame:
    """
    Evaluate model predictions against ground truth.
    
    Args:
        df: DataFrame with 'precursors' and 'products' columns
        predictor: ReactionT5Predictor instance
        n_best: Number of predictions per reaction
        batch_size: Batch size for inference
    
    Returns:
        DataFrame with predictions and evaluation metrics
    """
    # Prepare reactions for batch prediction
    reactions = [{'reactants': row['precursors'], 'reagents': ' '} 
                 for _, row in df.iterrows()]
    
    # Get predictions
    all_predictions = predictor.predict_batch(
        reactions, 
        batch_size=batch_size,
        num_beams=n_best,
        num_return_sequences=n_best
    )
    
    # Add predictions to DataFrame
    results = df.copy()
    results['target_canonical'] = results['products'].apply(canonicalize_smiles)
    
    # Store predictions
    for i in range(n_best):
        results[f'pred_{i+1}'] = [preds[i]['canonical'] if i < len(preds) else '' 
                                   for preds in all_predictions]
        results[f'pred_{i+1}_valid'] = [preds[i]['valid'] if i < len(preds) else False 
                                         for preds in all_predictions]
    
    # Calculate rank of correct prediction
    def get_rank(row):
        target = row['target_canonical']
        for i in range(1, n_best + 1):
            if row[f'pred_{i}'] == target:
                return i
        return 0  # Not found
    
    results['rank'] = results.apply(get_rank, axis=1)
    
    return results

# Run evaluation (this may take a few minutes)
print(f"Evaluating on {len(val_df)} reactions...")
print("(This may take a few minutes)")

eval_results = evaluate_predictions(val_df, forward_predictor, n_best=5, batch_size=8)

# Calculate metrics
n_total = len(eval_results)
print("\n" + "=" * 50)
print("EVALUATION RESULTS")
print("=" * 50)

for k in [1, 2, 3, 5]:
    correct = (eval_results['rank'] > 0) & (eval_results['rank'] <= k)
    accuracy = correct.sum() / n_total * 100
    print(f"Top-{k} Accuracy: {accuracy:.1f}%")

# Invalid SMILES rate
invalid_rate = (~eval_results['pred_1_valid']).sum() / n_total * 100
print(f"\nInvalid Top-1 SMILES: {invalid_rate:.1f}%")

In [ ]:
# Cell 12: Visualize Evaluation Results

# Create accuracy bar chart
accuracies = []
for k in [1, 2, 3, 4, 5]:
    correct = (eval_results['rank'] > 0) & (eval_results['rank'] <= k)
    accuracy = correct.sum() / len(eval_results) * 100
    accuracies.append({'k': f'Top-{k}', 'accuracy': accuracy})

acc_df = pd.DataFrame(accuracies)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=acc_df['k'],
    y=acc_df['accuracy'],
    marker_color=GOMES_COLORS['teal'],
    text=[f"{a:.1f}%" for a in acc_df['accuracy']],
    textposition='outside'
))

fig.update_layout(
    template=GOMES_TEMPLATE,
    title='ReactionT5v2 Accuracy on USPTO Validation Set',
    xaxis_title='',
    yaxis_title='Accuracy (%)',
    yaxis_range=[0, 100],
    showlegend=False,
    height=400,
)

fig.show()

<a id='2.4'></a>
### 2.4 Retrosynthesis Prediction

Retrosynthesis is the reverse problem: given a target product, predict the reactants needed to synthesize it. This is crucial for drug discovery and chemical manufacturing.

ReactionT5v2 also provides a retrosynthesis model.

In [ ]:
# Cell 13: Retrosynthesis Predictor

class RetrosynthesisPredictor:
    """
    Wrapper for ReactionT5v2 retrosynthesis model.
    
    Given a target product, predicts possible reactants.
    """
    
    def __init__(self, model_name: str = "sagawa/ReactionT5v2-retrosynthesis", device: str = None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model_name = model_name
        
        print(f"Loading retrosynthesis model: {model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.model.to(self.device)
        self.model.eval()
        print("Model loaded!")
    
    def predict(self, product: str, num_beams: int = 10, 
                num_return_sequences: int = 5) -> List[Dict]:
        """
        Predict reactants for a target product.
        
        Args:
            product: SMILES of target product
            num_beams: Beam search width
            num_return_sequences: Number of predictions
        
        Returns:
            List of predicted reactant sets
        """
        input_text = f"PRODUCT:{product}"
        inputs = self.tokenizer(input_text, return_tensors="pt", padding=True)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                num_beams=num_beams,
                num_return_sequences=num_return_sequences,
                max_length=200,
            )
        
        results = []
        for seq in outputs:
            pred = self.tokenizer.decode(seq, skip_special_tokens=True)
            pred = pred.replace(' ', '').rstrip('.')
            
            # Validate each reactant
            reactants = pred.split('.')
            valid = all(canonicalize_smiles(r) != '' for r in reactants if r)
            
            results.append({
                'reactants': pred,
                'valid': valid,
            })
        
        return results

# Load retrosynthesis model
print("Loading retrosynthesis model...")
retro_predictor = RetrosynthesisPredictor(device=DEVICE)

In [ ]:
# Cell 14: Retrosynthesis Examples

# Target molecules for retrosynthesis
targets = [
    {
        "name": "Aspirin",
        "smiles": "CC(=O)Oc1ccccc1C(=O)O",
    },
    {
        "name": "Ibuprofen",
        "smiles": "CC(C)Cc1ccc(C(C)C(=O)O)cc1",
    },
    {
        "name": "Ethyl Acetate",
        "smiles": "CCOC(C)=O",
    },
]

print("Retrosynthesis Predictions")
print("=" * 70)

for target in targets:
    print(f"\nTarget: {target['name']}")
    print(f"SMILES: {target['smiles']}")
    
    # Show target molecule
    mol = Chem.MolFromSmiles(target['smiles'])
    display(mol)
    
    # Get retrosynthesis predictions
    predictions = retro_predictor.predict(target['smiles'], num_return_sequences=3)
    
    print("\nPredicted Reactants:")
    for i, pred in enumerate(predictions, 1):
        valid_str = "valid" if pred['valid'] else "INVALID"
        print(f"  {i}. {pred['reactants']} ({valid_str})")
    print()

---

## Part 3: Advanced Topics

<a id='3.1'></a>
### 3.1 Atom Mapping with RXNMapper

Atom mapping shows which atoms in the reactants correspond to which atoms in the products. This is crucial for understanding reaction mechanisms.

[RXNMapper](https://github.com/rxn4chemistry/rxnmapper) uses transformer attention to learn atom correspondences without supervision.

In [ ]:
# Cell 15: Install and Setup RXNMapper

# RXNMapper installation (may require specific handling for Colab)
try:
    from rxnmapper import RXNMapper
    RXNMAPPER_AVAILABLE = True
    print("RXNMapper is available!")
except ImportError:
    print("Installing RXNMapper...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rxnmapper"])
    
    try:
        from rxnmapper import RXNMapper
        RXNMAPPER_AVAILABLE = True
        print("RXNMapper installed successfully!")
    except ImportError as e:
        RXNMAPPER_AVAILABLE = False
        print(f"RXNMapper installation failed: {e}")
        print("This is optional - the notebook will continue without atom mapping.")

In [ ]:
# Cell 16: Atom Mapping Examples

if RXNMAPPER_AVAILABLE:
    # Initialize RXNMapper
    rxn_mapper = RXNMapper()
    
    # Example reactions for atom mapping
    reactions_to_map = [
        "CC(=O)O.CCO>>CCOC(C)=O.O",  # Esterification
        "Brc1ccccc1.OB(O)c1ccccc1>>c1ccc(-c2ccccc2)cc1",  # Suzuki coupling
        "c1ccccc1.Cl>>Clc1ccccc1",  # Chlorination
    ]
    
    print("Atom Mapping Examples")
    print("=" * 70)
    
    for rxn in reactions_to_map:
        print(f"\nReaction: {rxn}")
        
        # Get atom mapping
        result = rxn_mapper.get_attention_guided_atom_maps([rxn])[0]
        mapped_rxn = result['mapped_rxn']
        confidence = result['confidence']
        
        print(f"Mapped:   {mapped_rxn}")
        print(f"Confidence: {confidence:.3f}")
        
        # Draw the mapped reaction
        try:
            display(SVG(draw_chemical_reaction(mapped_rxn, use_smiles=True)))
        except:
            print("(Could not render reaction image)")
else:
    print("RXNMapper not available. Skipping atom mapping examples.")
    print("You can still use the model for reaction prediction.")

<a id='3.2'></a>
### 3.2 IBM RXN for Chemistry API

[IBM RXN for Chemistry](https://rxn.res.ibm.com) provides access to production-grade reaction prediction models through a REST API. This is useful for:

- Access to models trained on larger datasets
- Production applications
- No local GPU required

**Note:** You need a free API key from [rxn.res.ibm.com](https://rxn.res.ibm.com).

In [ ]:
# Cell 17: IBM RXN for Chemistry Setup (Optional)

# Install rxn4chemistry client
try:
    from rxn4chemistry import RXN4ChemistryWrapper
    RXN4CHEM_AVAILABLE = True
except ImportError:
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rxn4chemistry"])
    try:
        from rxn4chemistry import RXN4ChemistryWrapper
        RXN4CHEM_AVAILABLE = True
    except:
        RXN4CHEM_AVAILABLE = False

if RXN4CHEM_AVAILABLE:
    print("rxn4chemistry is available!")
    print("")
    print("To use IBM RXN for Chemistry:")
    print("1. Create a free account at https://rxn.res.ibm.com")
    print("2. Get your API key from your profile settings")
    print("3. Set it below:")
    print("")
    print("# Example usage:")
    print("# api_key = 'YOUR_API_KEY_HERE'")
    print("# rxn = RXN4ChemistryWrapper(api_key=api_key)")
    print("# rxn.create_project('my_project')")
    print("# rxn.predict_reaction('BrBr.c1ccc2cc3ccccc3cc2c1')")
else:
    print("rxn4chemistry not available.")

In [ ]:
# Cell 18: IBM RXN API Example (requires API key)

# Uncomment and fill in your API key to use
USE_IBM_RXN = False  # Set to True if you have an API key

if USE_IBM_RXN and RXN4CHEM_AVAILABLE:
    # Replace with your actual API key
    api_key = "YOUR_API_KEY_HERE"  
    
    rxn = RXN4ChemistryWrapper(api_key=api_key)
    rxn.create_project('reaction_prediction_demo')
    
    # Predict a reaction
    reactants = "CC(=O)O.CCO"  # Acetic acid + Ethanol
    result = rxn.predict_reaction(reactants)
    
    print(f"Input: {reactants}")
    print(f"Predicted products: {result}")
else:
    print("IBM RXN API demo skipped.")
    print("Set USE_IBM_RXN = True and provide your API key to test.")

<a id='3.3'></a>
### 3.3 Data Augmentation for SMILES

A key technique for improving chemical language models is **SMILES randomization**. Since the same molecule can be written as many different valid SMILES strings, we can augment training data by generating alternative representations.

This teaches the model that different SMILES can represent the same molecule, improving generalization.

In [ ]:
# Cell 19: SMILES Randomization Functions

def randomize_smiles(smiles: str, random_type: str = "rotated") -> Optional[str]:
    """
    Generate a random but equivalent SMILES representation.
    
    Three randomization strategies:
    - 'rotated': Change the starting atom (preserves ring-opening order)
    - 'restricted': Shuffle atom order with some constraints
    - 'unrestricted': Full random atom ordering
    
    Args:
        smiles: Input canonical SMILES
        random_type: Randomization strategy
    
    Returns:
        Randomized SMILES or None if invalid
    """
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None
    
    if random_type == "unrestricted":
        # Full random SMILES (most diverse)
        return Chem.MolToSmiles(mol, canonical=False, doRandom=True, isomericSmiles=True)
    
    elif random_type == "restricted":
        # Random atom order with some constraints
        new_atom_order = list(range(mol.GetNumAtoms()))
        random.shuffle(new_atom_order)
        random_mol = Chem.RenumberAtoms(mol, newOrder=new_atom_order)
        return Chem.MolToSmiles(random_mol, canonical=False, isomericSmiles=True)
    
    elif random_type == "rotated":
        # Rotate starting atom (least disruptive)
        n_atoms = mol.GetNumAtoms()
        rotation_index = random.randint(0, n_atoms - 1)
        atoms = list(range(n_atoms))
        new_order = atoms[rotation_index:] + atoms[:rotation_index]
        rotated_mol = Chem.RenumberAtoms(mol, new_order)
        return Chem.MolToSmiles(rotated_mol, canonical=False, isomericSmiles=True)
    
    raise ValueError(f"Unknown random_type: {random_type}")

# Demonstrate SMILES randomization
caffeine = "Cn1cnc2c1c(=O)n(c(=O)n2C)C"  # Caffeine
canonical = canonicalize_smiles(caffeine)

print(f"Canonical SMILES: {canonical}")
print("\nRandomized versions:")

for rand_type in ["rotated", "restricted", "unrestricted"]:
    print(f"\n{rand_type.upper()}:")
    unique_smiles = set()
    for _ in range(100):
        rand_smi = randomize_smiles(canonical, rand_type)
        if rand_smi:
            unique_smiles.add(rand_smi)
    
    print(f"  Generated {len(unique_smiles)} unique SMILES from 100 attempts")
    print(f"  Examples: {list(unique_smiles)[:3]}")

# Verify all random SMILES canonicalize to the same molecule
print("\nValidation: All random SMILES should canonicalize to the same molecule")
test_smiles = [randomize_smiles(canonical, "unrestricted") for _ in range(10)]
canonicalized = set(canonicalize_smiles(s) for s in test_smiles if s)
print(f"Unique canonical forms: {canonicalized}")
assert len(canonicalized) == 1, "Error: Different molecules produced!"

In [ ]:
# Cell 20: Data Augmentation for Reaction Datasets

def augment_reaction_dataset(df: pd.DataFrame, n_augmentations: int = 1,
                              random_type: str = "rotated") -> pd.DataFrame:
    """
    Augment a reaction dataset with randomized SMILES.
    
    Each reaction is copied with randomized precursor SMILES.
    Products are kept canonical (as targets should be deterministic).
    
    Args:
        df: DataFrame with 'precursors' and 'products' columns
        n_augmentations: Number of augmented copies per reaction
        random_type: SMILES randomization strategy
    
    Returns:
        Augmented DataFrame
    """
    augmented_data = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Augmenting"):
        # Original reaction
        augmented_data.append({
            'precursors': row['precursors'],
            'products': row['products'],
            'is_augmented': False
        })
        
        # Augmented copies
        for _ in range(n_augmentations):
            # Randomize each reactant separately
            reactants = row['precursors'].split('.')
            rand_reactants = []
            for r in reactants:
                rand_r = randomize_smiles(r, random_type)
                rand_reactants.append(rand_r if rand_r else r)
            
            augmented_data.append({
                'precursors': '.'.join(rand_reactants),
                'products': row['products'],
                'is_augmented': True
            })
    
    return pd.DataFrame(augmented_data)

# Demonstrate augmentation on a small subset
sample_df = val_df.head(5)
augmented_df = augment_reaction_dataset(sample_df, n_augmentations=2)

print(f"Original dataset: {len(sample_df)} reactions")
print(f"Augmented dataset: {len(augmented_df)} reactions")
print(f"Augmentation factor: {len(augmented_df) / len(sample_df):.1f}x")
print("\nSample augmented data:")
augmented_df.head(9)

<a id='3.4'></a>
### 3.4 Fine-tuning on Custom Datasets

If you have a specialized reaction type (e.g., enzymatic reactions, photochemistry), you can fine-tune ReactionT5 on your own data. The model shows strong performance even with small datasets (~100-200 reactions).

In [ ]:
# Cell 21: Fine-tuning Setup (Conceptual)

# This cell shows the structure for fine-tuning.
# For actual fine-tuning, you would need to prepare your dataset and run training.

print("""
Fine-tuning ReactionT5v2 on Custom Data
========================================

Step 1: Prepare your data as a CSV file with columns:
  - REACTANT: SMILES of reactants (multiple compounds joined with '.')
  - REAGENT: SMILES of reagents/catalysts (or ' ' if none)
  - PRODUCT: SMILES of products

Step 2: Clone the ReactionT5v2 repository:
  git clone https://github.com/sagawatatsuya/ReactionT5v2
  cd ReactionT5v2

Step 3: Run fine-tuning:
  cd task_forward
  python finetune.py \\
      --model_name_or_path='sagawa/ReactionT5v2-forward' \\
      --epochs=50 \\
      --batch_size=32 \\
      --train_data_path='your_train.csv' \\
      --valid_data_path='your_valid.csv' \\
      --output_dir='output'

Tips for fine-tuning:
  - Start with a small learning rate (1e-4 to 1e-5)
  - Use early stopping based on validation loss
  - Augment small datasets with SMILES randomization
  - Consider transfer learning from related reaction types
""")

---

## Part 4: Visualization and Analysis

<a id='4.1'></a>
### 4.1 Drawing Chemical Reactions

In [ ]:
# Cell 22: Enhanced Reaction Drawing

def draw_reaction_comparison(precursors: str, predicted: str, expected: str,
                             title: str = "Reaction Comparison") -> None:
    """
    Draw predicted vs expected reaction products side by side.
    
    Args:
        precursors: SMILES of reactants
        predicted: Predicted product SMILES
        expected: Ground truth product SMILES
        title: Display title
    """
    from IPython.display import display, HTML
    
    pred_canonical = canonicalize_smiles(predicted)
    exp_canonical = canonicalize_smiles(expected)
    is_correct = pred_canonical == exp_canonical
    
    status_color = GOMES_COLORS['success'] if is_correct else GOMES_COLORS['coral']
    status_text = "CORRECT" if is_correct else "INCORRECT"
    
    html = f"""
    <div style="border: 2px solid {status_color}; padding: 15px; margin: 10px 0; border-radius: 8px;">
        <h3 style="color: {GOMES_COLORS['navy']};">{title}</h3>
        <p><strong>Status:</strong> <span style="color: {status_color}; font-weight: bold;">{status_text}</span></p>
        <p><strong>Reactants:</strong> <code>{precursors}</code></p>
        <p><strong>Predicted:</strong> <code>{predicted}</code></p>
        <p><strong>Expected:</strong> <code>{expected}</code></p>
    </div>
    """
    display(HTML(html))
    
    # Draw the predicted reaction
    rxn_smiles = f"{precursors}>>{predicted}"
    try:
        display(SVG(draw_chemical_reaction(rxn_smiles)))
    except:
        print("(Could not render reaction)")

# Show some examples from our evaluation
print("Sample Predictions from Evaluation")
print("=" * 50)

# Show correct predictions
correct_samples = eval_results[eval_results['rank'] == 1].head(3)
for _, row in correct_samples.iterrows():
    draw_reaction_comparison(
        row['precursors'],
        row['pred_1'],
        row['target_canonical'],
        "Correct Prediction"
    )

# Show incorrect predictions
incorrect_samples = eval_results[eval_results['rank'] == 0].head(2)
for _, row in incorrect_samples.iterrows():
    draw_reaction_comparison(
        row['precursors'],
        row['pred_1'],
        row['target_canonical'],
        "Incorrect Prediction"
    )

<a id='4.2'></a>
### 4.2 Error Analysis and Debugging

Understanding why predictions fail is crucial for improving models and understanding their limitations.

In [ ]:
# Cell 23: Error Analysis

def analyze_prediction_errors(results_df: pd.DataFrame) -> Dict:
    """
    Analyze types of prediction errors.
    
    Categories:
    - Invalid SMILES: Model output couldn't be parsed
    - Wrong product: Valid SMILES but incorrect
    - Partial match: Correct in later predictions (Top-2 to Top-5)
    
    Args:
        results_df: DataFrame from evaluate_predictions
    
    Returns:
        Dictionary with error statistics
    """
    n_total = len(results_df)
    
    # Categorize errors
    correct_top1 = (results_df['rank'] == 1).sum()
    correct_top5 = (results_df['rank'] > 0).sum()
    invalid_smiles = (~results_df['pred_1_valid']).sum()
    wrong_but_valid = (results_df['rank'] == 0) & (results_df['pred_1_valid'])
    
    analysis = {
        'total_reactions': n_total,
        'correct_top1': correct_top1,
        'correct_top1_pct': correct_top1 / n_total * 100,
        'correct_top5': correct_top5,
        'correct_top5_pct': correct_top5 / n_total * 100,
        'invalid_smiles': invalid_smiles,
        'invalid_smiles_pct': invalid_smiles / n_total * 100,
        'wrong_but_valid': wrong_but_valid.sum(),
        'wrong_but_valid_pct': wrong_but_valid.sum() / n_total * 100,
    }
    
    return analysis

# Run error analysis
error_analysis = analyze_prediction_errors(eval_results)

print("Error Analysis")
print("=" * 50)
print(f"Total reactions evaluated: {error_analysis['total_reactions']}")
print(f"")
print(f"Correct (Top-1): {error_analysis['correct_top1']} ({error_analysis['correct_top1_pct']:.1f}%)")
print(f"Correct (Top-5): {error_analysis['correct_top5']} ({error_analysis['correct_top5_pct']:.1f}%)")
print(f"Invalid SMILES: {error_analysis['invalid_smiles']} ({error_analysis['invalid_smiles_pct']:.1f}%)")
print(f"Wrong but valid: {error_analysis['wrong_but_valid']} ({error_analysis['wrong_but_valid_pct']:.1f}%)")

In [ ]:
# Cell 24: Common Failure Patterns

print("""
Common Failure Patterns in Reaction Prediction
===============================================

1. STEREOCHEMISTRY ERRORS
   - Model predicts correct connectivity but wrong stereochemistry
   - Often seen in reactions involving chiral centers
   - The USPTO dataset lacks stereochemistry, so models may struggle

2. REGIOSELECTIVITY ERRORS
   - Model predicts reaction at wrong position on molecule
   - Common for aromatic substitution reactions
   - Example: ortho vs para product in electrophilic substitution

3. INCOMPLETE REACTIONS
   - Model predicts only partial transformation
   - May miss side products or byproducts
   - Common for multi-step reactions treated as single step

4. OVER-GENERALIZATION
   - Model predicts common reaction type inappropriately
   - May suggest Suzuki coupling when conditions don't support it
   - Result of training data bias

5. INVALID SMILES
   - Model generates syntactically invalid output
   - Often happens with very long molecules
   - May indicate out-of-distribution inputs

Debugging Tips:
   - Always validate output SMILES with RDKit
   - Check if reactants contain unusual functional groups
   - Consider whether reaction type was in training data
   - Use beam search (multiple predictions) to see alternatives
""")

---

<a id='appendix-a'></a>
## Appendix A: Legacy OpenNMT-py Approach (Reference Only)

The original 2022 notebook used OpenNMT-py for training Molecular Transformers. While this approach is now deprecated, understanding the architecture is still valuable.

**Why OpenNMT-py is no longer recommended:**

1. **Maintenance Mode**: As of July 2024, OpenNMT-py is no longer actively developed. The successor project is [Eole](https://github.com/eole-nlp/eole).

2. **Dependency Issues**: The `pyonmttok` tokenizer doesn't support Python 3.12 (Google Colab's current version).

3. **Better Alternatives**: HuggingFace Transformers offers better ecosystem integration, more models, and active development.

For historical reference, the key OpenNMT-py commands were:

```bash
# Build vocabulary
onmt_build_vocab -config config.yaml -n_sample -1

# Train model
onmt_train -config config.yaml -seed 42 -gpu_ranks 0 ...

# Translate (predict)
onmt_translate -model model.pt -src test_src.txt -output predictions.txt
```

<a id='appendix-b'></a>
## Appendix B: Publications and Resources

### Core Papers

**Molecular Transformer:**
- Schwaller, P. et al. "Molecular Transformer: A Model for Uncertainty-Calibrated Chemical Reaction Prediction" *ACS Central Science* (2019) [Link](https://pubs.acs.org/doi/10.1021/acscentsci.9b00576)

**ReactionT5:**
- Sagawa, T. & Kojima, R. "ReactionT5: a pre-trained transformer model for accurate chemical reaction prediction with limited data" *Journal of Cheminformatics* (2025) [Link](https://doi.org/10.1186/s13321-025-01075-4)

**RXNMapper:**
- Schwaller, P. et al. "Extraction of organic chemistry grammar from unsupervised learning of chemical reactions" *Science Advances* (2021) [Link](https://www.science.org/doi/10.1126/sciadv.abe4166)

### Additional Resources

**Tutorials and Blogs:**
- [Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/) - Visual explanation of transformer architecture
- [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html) - Code walkthrough

**Tools and Datasets:**
- [Open Reaction Database (ORD)](https://open-reaction-database.org/) - Large-scale reaction data
- [IBM RXN for Chemistry](https://rxn.res.ibm.com) - Production API
- [RDKit](https://www.rdkit.org/) - Cheminformatics toolkit

**Review Papers:**
- Schwaller, P. "Machine Intelligence for Chemical Reaction Space" *WIREs Computational Molecular Science* (2022) [Link](https://wires.onlinelibrary.wiley.com/doi/full/10.1002/wcms.1604)

In [ ]:
# Cell 25: Session Summary

print("""
================================================================================
                    SESSION SUMMARY: REACTION PREDICTION
================================================================================

Key Takeaways:

1. SMILES as Chemical Language
   - Molecules and reactions can be represented as text strings
   - Canonicalization ensures consistent representation
   - Atom-wise tokenization preserves chemical meaning

2. Transformer Models for Chemistry
   - Treat reaction prediction as sequence-to-sequence translation
   - Self-attention captures long-range dependencies in molecules
   - Pre-training on large datasets enables transfer learning

3. ReactionT5v2 Advantages
   - Pre-trained on diverse Open Reaction Database
   - Easy fine-tuning for specialized applications
   - Works out-of-the-box on Google Colab

4. Practical Considerations
   - Always validate output SMILES
   - Use beam search for multiple predictions
   - Data augmentation improves generalization
   - Consider API services for production use

Next Steps:
   - Try fine-tuning on your own reaction dataset
   - Explore retrosynthesis for synthesis planning
   - Integrate with reaction fingerprints (RXNFP/DRFP)
   - Investigate yield prediction capabilities

================================================================================
""")

In [ ]:
# Cell 26: Clean up GPU memory
import gc

# Clean up models to free memory
if 'forward_predictor' in dir():
    del forward_predictor
if 'retro_predictor' in dir():
    del retro_predictor

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU memory cleared. Current usage: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
else:
    print("Session complete.")